In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import IPython
from tqdm import tqdm
from PIL import Image
import scipy.io
tf.compat.v1.enable_eager_execution()
print(tf.executing_eagerly())
"""DO NOT FORGET TO PASTE THE ABOVE LINE TODO"""
mat = scipy.io.loadmat(os.getcwd() + '\CCp\\annotations\pixel-level\\0004.mat')
im = Image.fromarray(mat['groundtruth'])
im2 = Image.open(os.getcwd() + '\CCp\\photos\\0004.jpg')
# im_anno = scipy.io.loadmat(os.getcwd() + '\CCp\\annotations\image-level\\1004.mat')

E:\Applications\Anaconda\envs\Twitter\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Applications\Anaconda\envs\Twitter\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Applications\Anaconda\envs\Twitter\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Applications\Anaconda\envs\Twitter\lib\site-packages\tensorflow\

True


In [20]:
color_map = {}
import pickle
import random
color_map['0'] = (0,0,0,255)
random.seed(20)
for i in range(150, 256):
    color_map[f'{i}'] = [random.randint(0,255), random.randint(0,255), random.randint(0,255), 255]

pickle.dump(color_map, open("colormap.bin", 'wb'))
# print(pickle.load(open("colormap.bin", 'rb')))
    

In [22]:


def color_helper(pix):
    # print(color_map[f'{pix}'])
    return color_map[f'{pix}']

def colormap_pat(nparray):
    new_image_array = []
    for row in nparray:
        temp_col = []
        for column in row:
            temp_col.append(color_helper(column))
            # print(column, color_helper(column), cm.flag(column))
            
        new_image_array.append(temp_col)
        
        
    return new_image_array

from PIL import Image
from matplotlib import cm
im = Image.fromarray(np.uint8(colormap_pat(mat['groundtruth']*255)))
# im = im.convert('RGB')


In [13]:
im.size

(550, 810)

In [23]:
def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result



In [24]:
images = os.listdir(os.getcwd()  + '\CCp\\annotations\pixel-level\\' )

In [25]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

In [26]:
path  =os.getcwd() + "\CCp\data\\" 
dims = set()
for folder in os.listdir(path):
    for image in os.listdir(path + folder):
        # print(path+folder+"\\" +image)
        size = Image.open(path+folder+"\\" +image).size
        dims.add(size)
        
    
print(len(dims))

1


In [27]:
im = Image.fromarray(np.uint8(mat['groundtruth']*255))

# print(im.__array__())
catalogue = []
for i,image_name in enumerate(tqdm(images)):
    mat = scipy.io.loadmat(os.getcwd() + f'\CCp\\annotations\pixel-level\\{image_name}')
    

    img_array = np.uint8(colormap_pat(mat['groundtruth']*255))
    image = Image.fromarray(img_array)
    dimensions = image.size

    im1 = image.resize((564, 832), 0)

    
    im1 = add_margin(im1, 0,134,0,134, (0,0,0))
    
    im2 = Image.open(os.getcwd() + f'\CCp\\photos\\{image_name[:-4]}.jpg')
    # im2 = add_margin(im2, 0, 141, 0, 141, (0,0,0))
    dimensions = im2.size
    im2 = im2.resize((564, 832), 0)
    im2 = add_margin(im2, 0,134,0,134, (0,0,0))
    

    # print(im1.size)
    assert im1.size == (832, 832)
    assert im2.size == (832, 832)
    image = get_concat_h(im2, im1)
    # image.save("FORTNITES.PNG")
  
    if i < int(len(images) * 0.95):
        savePath = os.getcwd() + f'\CCp\data\\train\\{image_name[:-4]}.png'
    elif i < int(len(images) * 0.99):
        savePath = os.getcwd() + f'\CCp\data\\test\\{image_name[:-4]}.png'
    else:
        savePath = os.getcwd() + f'\CCp\data\\val\\{image_name[:-4]}.png'
    image.save(savePath)
    # load in actual image
    
    

print(catalogue)

  0%|          | 4/1004 [00:07<29:54,  1.79s/it]


KeyboardInterrupt: 

In [4]:
im = Image.fromarray(np.uint8(mat['groundtruth']*255))

# print(im.__array__())
catalogue = []
for i,image_name in tqdm(enumerate(images)):
    mat = scipy.io.loadmat(os.getcwd() + f'\CCp\\annotations\pixel-level\\{image_name}')
    

    img_array = np.uint8(colormap_pat(mat['groundtruth']*255))
    image = Image.fromarray(img_array)
    im1 = add_margin(image, 0, 141, 0, 141, (0,0,0))
    
    im2 = Image.open(os.getcwd() + f'\CCp\\photos\\{image_name[:-4]}.jpg')
    im2 = add_margin(im2, 0, 141, 0, 141, (0,0,0))
    image = get_concat_h(im2, im1)
    
    savePath = os.getcwd() + f'\CCp\data\\{image_name[:-4]}TEST.jpeg'
    image.save(savePath)

    break
    if i < int(len(images) * 0.95):
        savePath = os.getcwd() + f'\CCp\data\\train\\{image_name[:-4]}.jpg'
        # break
    elif i < int(len(images) * 0.99):
        savePath = os.getcwd() + f'\CCp\data\\test\\{image_name[:-4]}.png'
    else:
        savePath = os.getcwd() + f'\CCp\data\\val\\{image_name[:-4]}.png'
    image.save(savePath)
    #load in actual image
    
    

print(catalogue)

NameError: name 'images' is not defined

In [5]:
import tensorflow as tf
from IPython import display
import tensorflow.compat.v1 as tf1
# tf1.config.experimental_run_functions_eagerly(True)

def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image)

    w = tf.shape(image)[1]

    w = w // 2
    real_image = image[:, :w, :]
    input_image = image[:, w:, :]

    input_image = tf.cast(input_image, tf.float32)
    real_image = tf.cast(real_image, tf.float32)

    return input_image, real_image

    """
    OG Function
    """


def resize(input_image, real_image, height, width):
    input_image = tf.image.resize(input_image, [height, width],
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    real_image = tf.image.resize(real_image, [height, width],
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return input_image, real_image


def random_crop(input_image, real_image):
    stacked_image = tf.stack([input_image, real_image], axis=0)
    cropped_image = tf.image.random_crop(stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image[0], cropped_image[1]


# normalizing the images between [-1, 1]

def normalize(input_image, real_image):
    input_image = (input_image / 127.5) - 1
    real_image = (real_image / 127.5) - 1

    return real_image, input_image


def add_noise(input_image, real_image):
    
    
    return input_image, real_image

@tf.function()
def random_jitter(input_image, real_image):
    # resizing to 286 x 286 x 3
    input_image, real_image = resize(input_image, real_image, 286, 286)

    # randomly cropping to 256 x 256 x 3
    input_image, real_image = random_crop(input_image, real_image)

    #add random color
#     input_image, real_image = add_noise(input_image, real_image)
    
    
    if tf.random.uniform(()) > 0.5:
        # random mirroring
        input_image = tf.image.flip_left_right(input_image)
        real_image = tf.image.flip_left_right(real_image)

    return input_image, real_image


def load_image_train(image_file):
    print(image_file)
    input_image, real_image = load(image_file)
    input_image, real_image = random_jitter(input_image, real_image)
    input_image, real_image = normalize(input_image, real_image)

    return input_image, real_image


def load_image_test(image_file):
    input_image, real_image = load(image_file)
    input_image, real_image = resize(input_image, real_image,IMG_HEIGHT, IMG_WIDTH)
    input_image, real_image = normalize(input_image, real_image)

    return input_image, real_image

In [13]:
e = tf.make_tensor_proto(ins)

In [12]:
from IPython import display
# import tensorflow.compat.v1 as tf1
# tf1.config.experimental_run_functions_eagerly(False)

import tensorflow as tf
import numpy as np

# tf.compat.v1.disable_eager_execution()

def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image)

    w = tf.shape(image)[1]

    w = w // 2
    real_image = image[:, :w, :]
    input_image = image[:, w:, :]

    input_image = tf.cast(input_image, tf.float32)
    real_image = tf.cast(real_image, tf.float32)

    return input_image, real_image

    """
    OG Function
    """




from PIL import Image
print(os.getcwd())
ins, real = load(os.getcwd() + '/CCp/data/0001TEST.jpeg')
ins, real = normalize(ins, real)

tf.ensure_shape(ins, [832,832,3])
# ins.shape = [832,832,3]
print(ins.shape)
print(tf.executing_eagerly())
# image = tf.keras.preprocessing.image.array_to_img(ins)
# display(image)
im = tf.make_ndarray(tf.make_tensor_proto(ins))

im = Image.fromarray(im)
display(im)

c:\Users\Patrick\Documents\GH_REPOS\POLY-GAN
(832, 832, 3)
True


TypeError: Cannot handle this data type: (1, 1, 3), <f4

In [74]:
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])

print(tf.executing_eagerly())
n = c.numpy()

ValueError: tf.enable_eager_execution must be called at program startup.